In [1]:
# Imports and data initialization
from data.orchestrator import initialize
from pricing.cds_pricing import par_spread, price_cds

data = initialize()

# CDS Pricing Test: Par Spread and PV at Par
This notebook demonstrates how to compute the par spread and present value (PV) for a 5-year CDS contract using a quarterly survival grid (step = 0.25 years).

In [2]:
# Set Parameters for Test
T = 5.0
survival_grid_step = 0.25

In [3]:
# Calculate Par Spread
s_par = par_spread(data, maturity=T, survival_grid_step=survival_grid_step)

TypeError: par_spread() got an unexpected keyword argument 'survival_grid_step'

In [4]:
# Price CDS at Par Spread
pv = price_cds(data, maturity=T, spread=s_par, notional=1.0, survival_grid_step=survival_grid_step)

NameError: name 's_par' is not defined

In [6]:
# Display Results
print(f"Par spread 5Y: {s_par:.8f}")
print(f"PV(buyer) at par spread: {pv:.10f}")
# Ideal output:
# Par spread 5Y: 0.01234567
# PV(buyer) at par spread: 0.0000000000  (or very close, e.g. < 1e-8)

NameError: name 's_par' is not defined

In [9]:
# All-in-one CDS bucket sensitivity test cell

from pricing.cds_pricing import compute_cds_sensitivities, bump_forward_hazard, price_cds

# Compute base spreads, deltas, and base PVs for 5Y CDS
base_spreads, deltas = compute_cds_sensitivities(
    data, bump=0.0010, maturities=(5.0,), pay_freq=4, include_accrual=True, protection_buyer=True
)
base_pv = {}
for T in base_spreads:
    base_pv[T] = price_cds(
        data, maturity=T, spread=base_spreads[T], notional=1.0,
        pay_freq=4, include_accrual=True, protection_buyer=True,
    )

# Pick one bucket to inspect (e.g. the first one)
example_bucket = list(deltas.keys())[0]   # e.g. '0_1'

# Re-create bumped data for that bucket
bumped_data = bump_forward_hazard(data, example_bucket, bump=0.0010)

T = 5.0
s = base_spreads[T]
pv_base = base_pv[T]
pv_bumped = price_cds(
    bumped_data, maturity=T, spread=s, notional=1.0,
    pay_freq=4, include_accrual=True, protection_buyer=True, 
)

print(f"Bucket: {example_bucket}")
print(f"Base PV     : {pv_base:12.8e}")
print(f"Bumped PV   : {pv_bumped:12.8e}")
print(f"Delta (ΔV)  : {pv_bumped - pv_base:12.8e}")

Bucket: 0_1
Base PV     : 0.00000000e+00
Bumped PV   : 3.88242684e-04
Delta (ΔV)  : 3.88242684e-04
